                                                            Google Maps Scrapper

In [8]:
from selenium import webdriver
#import pyautogui
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from geopy.geocoders import Nominatim
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# #from unshortenit import UnshortenI
# import openlocationcode as olc

# import pluscodes
# import geohash2
# import pygeohash as pgh


In [9]:
filename = "data4"
link = "https://www.google.com/maps/search/restaurants+in+Cairo/@30.0467159,31.2205123,11.96z/data=!4m2!2m1!6e5?entry=ttu/en" 
# browser = webdriver.Edge()
record = []
e = []
le = 0

# Set the language preference in the browser 
driver=webdriver.Edge()
#driver.get("https://www.google.com/maps/search/restaurants+in+Cairo/@30.0467159,31.2205123,11.96z/data=!4m2!2m1!6e5?entry=ttu")
print(driver.title)




def Selenium_extractor():
    

    # action = ActionChains(driver)
    # a = driver.find_elements(By.CLASS_NAME, "hfpxzc")

    langChangeStep1=driver.find_element(By.XPATH,""" //*[@id="QA0Szd"]/div/div/div[1]/div[1]/ul/li[1]/button """)
    langChangeStep1.click()
    time.sleep(2)
    langChangeStep2=driver.find_element(By.XPATH,"""//*[@id="settings"]/div/div[2]/ul/div[7]/li[1]/button""")
    langChangeStep2.click()
    time.sleep(2)
    langChangeStep3=driver.find_element(By.XPATH,""" //*[@id="languages"]/div/div[2]/div/div/a""")
    langChangeStep3.click()
    time.sleep(2)

    action = ActionChains(driver)
    a = driver.find_elements(By.CLASS_NAME, "hfpxzc")


    while len(a) < 20:
        print(len(a))
        var = len(a)
        scroll_origin = ScrollOrigin.from_element(a[len(a)-1])
        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
        time.sleep(2)
        a = driver.find_elements(By.CLASS_NAME, "hfpxzc")

        if len(a) == var:
            le+=1
            if le > 20:
                break
        else:
            le = 0
    print("----------------------")
    print("    Scrolling Done    ")
    print("----------------------")
    
    
    for i in range(len(a)):
        scroll_origin = ScrollOrigin.from_element(a[i])
        action.scroll_from_origin(scroll_origin, 0, 100).perform()
        action.move_to_element(a[i]).perform()
        a[i].click()
        time.sleep(2)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        # try:
        
        #name
        Name_Html = soup.findAll('h1', {"class": "DUwDvf lfPIob"})
        name = Name_Html[0].text
        if name not in e:
            e.append(name)
            divs = soup.findAll('div', {"class": "Io6YTe fontBodyMedium kR99db"})
            
            #Address
            Address=divs[0].text                                                                                       
            
            
            # for j in range(len(divs)):
            #         if str(divs[j].text)[0] == "+":
            #             phone = divs[j].text
            
            #phone number
            phoneFound=False
            
            while( phoneFound==False):
                for j in range(len(divs)):                                                                                 
                    for z in range(len(divs[j].text)):                                  #seeing every div from inside
                        if((str(divs[j].text)[z].isnumeric()) or (str(divs[j].text)[z] ==" ") or (str(divs[j].text)[z] == "+")):
                            if(z == (len(str(divs[j].text)))-1):                                           #does he tested all the char of the phone number or not
                                phone  = divs[j].text
                                phoneFound=True  
                            else:
                                continue
                        else:
                            if(phoneFound==False):
                                if(j == len(divs)-1):                                           # does he tested all the divs or not
                                    phone  = "Phone number not available"
                                    phoneFound=True                                            # to break the outer loop only (practically phone is not found )
                                else:  
                                    break
                             
                    
            #Website
            websiteFound=False
            try:                                                                                                         
                for z in range(len(divs)):
                    if str(divs[z].text)[-4] == "." or str(divs[z].text)[-3] == "." or str(divs[z].text)[-5] == ".":         # www.  , .com , .site
                        website = divs[z].text
                        websiteFound=True
                        
            except:
                if(websiteFound==True):
                    print("Website found")
                else:
                    website="Not available"  
        
            
            # rating
            Rating_Html = soup.findAll('div', {"class": "F7nice"})
            rating=Rating_Html[0].text[0:3]
            noOfRatings=Rating_Html[0].text[4:-1]
            
            
            #location link
            location_Html=driver.find_element(By.XPATH,""" //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[4]/div[5]/button """)
            location_Html.click()
            time.sleep(5)
            locationLink=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[3]/div/div/div[1]/div[4]/div[2]/input """).get_attribute("value")
            closeSharePanel=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[2]/button """)
            closeSharePanel.click()
            time.sleep(2)
            
            
            
            #plus code 
            counter=0
            plusCodeValue ="None"
            plusCodeFound=False   
            divPhotos=driver.find_elements(By.XPATH,"""//div[@class="cXHGnc"]""")    


            while(plusCodeFound==False):
                
                for z in range(len(divPhotos)):

                    t1="(//div[@class='cXHGnc']/div/img)["
                    t2=z+1
                    t3="]"
                    photoPath=str(t1)+str(t2)+str(t3)
                    Photo=driver.find_element(By.XPATH,str(photoPath) )
                    if(Photo.get_attribute("src")=="https://maps.gstatic.com/mapfiles/maps_lite/images/2x/ic_plus_code.png"):
                        plusCodeValue=divs[counter].text
                        plusCodeFound=True
                        break
                    else:
                        if(plusCodeFound==False):
                            if(z == len(divPhotos)-1):                                           # does he tested all the divs or not
                                plusCodeValue  = "not available"
                                plusCodeFound=True                                            # to break the outer loop only (practically phone is not found )
                            else:  
                                counter =counter+1
       
            p = driver.current_window_handle                #(save the last place in the main page)
        
            #Longitude and latitude (from plus code if found or from address if not found)
            if(plusCodeFound or Address):
                plusCodeToLongLatWebsite ="https://plus.codes/map"
                # Open a new window
                driver.execute_script("window.open('');")
                # Switch to the new window and open the website
                driver.switch_to.window(driver.window_handles[1])
                driver.get(plusCodeToLongLatWebsite)
                
                
                inputPlusCode = driver.find_element(By.XPATH, """ //*[@id="search-input"] """)
                if(plusCodeFound==True):
                    inputPlusCode.send_keys(plusCodeValue)
                    inputPlusCode.send_keys(Keys.ENTER)
                else:
                    inputPlusCode.send_keys(Address)
                    inputPlusCode.send_keys(Keys.ENTER)
                time.sleep(3)
                langChangeStep3=driver.find_element(By.XPATH,""" //*[@id="summary"]/div[1] """)
                langChangeStep3.click()
                
                time.sleep(5)
                
                longitudeAndLatitude =driver.find_element(By.XPATH,""" //*[@id="placecard-details"]/div[3]/div[1]  """).text
                latitude , longitude = longitudeAndLatitude.split(',')
                driver.close()
            else:
                continue
            driver.switch_to.window(p)
           
            print([name ,Address ,phone ,website , rating ,noOfRatings,locationLink,plusCodeValue ,latitude ,longitude])
            record.append((name ,Address ,phone ,website ,rating ,noOfRatings ,"Cairo",locationLink ,plusCodeValue ,latitude ,longitude))
            df=pd.DataFrame(record,columns=['Name' ,'Address' ,'Phone number' ,'Website' ,'Rating','Number of ratings' ,"Governorate","location Link","Plus code" ,"Latitude" ,"Longitude"])  # writing data to the file
            df.to_csv(filename + '.csv',index=False,encoding='utf-8')
        # except:
        #     print("error")
        #     continue




driver.get(str(link))
time.sleep(10)
Selenium_extractor()



5
10
15
----------------------
    Scrolling Done    
----------------------
['The Grill', 'Semiramis InterContinental, Cornich, El Nile, Cairo Governorate 11511', '02 27988000', 'Not available', '4.7', '686', 'https://maps.app.goo.gl/xYf5ACteBCsVZ3zy5', '26VJ+7P Qasr El Nil', '30.043188', '31.231813']


IndexError: list index out of range